# **Proyek Klasifikasi Gambar dengan CNN**

---

## **Lingkungan Pengembangan**
- **Bahasa Pemrograman**: Python
- **Framework**: TensorFlow dan Keras
- **Dataset**: Dataset gambar dengan kategori sampah

---

## **Tahapan Implementasi**


In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import shutil
from sklearn.model_selection import train_test_split

Tahap pertama adalah membagi dataset menjadi dua bagian utama:
- **Training Set**: Digunakan untuk melatih model, mencakup 80% dari total data.
- **Validation Set**: Digunakan untuk mengevaluasi performa model selama pelatihan, mencakup 20% dari total data.

Pembagian dataset dilakukan secara terstruktur ke dalam folder sesuai kategori, sehingga setiap kategori memiliki direktori terpisah untuk data pelatihan dan validasi. Struktur ini membantu dalam proses pemuatan data oleh generator gambar.


In [2]:
# Path dataset asli
original_dataset_dir = "dataset" 
output_train_dir = "dataset_split/train"
output_val_dir = "dataset_split/validation"

In [3]:
os.makedirs(output_train_dir, exist_ok=True)
os.makedirs(output_val_dir, exist_ok=True)

In [4]:
categories = os.listdir(original_dataset_dir)  # Folder kategori
for category in categories:
    category_path = os.path.join(original_dataset_dir, category)
    if not os.path.isdir(category_path):
        continue

    # Ambil semua file dalam kategori
    images = os.listdir(category_path)

    # Split dataset menjadi train dan validation (80%-20%)
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

    # Path output untuk setiap kategori
    train_category_dir = os.path.join(output_train_dir, category)
    val_category_dir = os.path.join(output_val_dir, category)

    # Membuat folder kategori di train dan validation
    os.makedirs(train_category_dir, exist_ok=True)
    os.makedirs(val_category_dir, exist_ok=True)

    # Pindahkan file ke folder train
    for image in train_images:
        src = os.path.join(category_path, image)
        dst = os.path.join(train_category_dir, image)
        shutil.copy(src, dst)

    # Pindahkan file ke folder validation
    for image in val_images:
        src = os.path.join(category_path, image)
        dst = os.path.join(val_category_dir, image)
        shutil.copy(src, dst)

print("Berhasil split dataset")

Berhasil split dataset


## **2. Preprocessing Dataset**
Setelah dataset dibagi, langkah berikutnya adalah melakukan preprocessing pada gambar. Beberapa langkah penting meliputi:
- **Rescaling**: Nilai piksel gambar dirubah menjadi skala 0-1 untuk mempercepat pelatihan.
- **Augmentasi Data**: Gambar dilatih dengan berbagai transformasi (rotasi, zoom, flipping) untuk meningkatkan kemampuan generalisasi model.

Hasil preprocessing ini dihasilkan oleh generator gambar seperti `ImageDataGenerator` yang mendukung augmentasi real-time selama pelatihan.

---

In [5]:
# tinggi
IMG_HEIGHT = 128
#lebar
IMG_WIDTH = 128
#jumlah gambar
BATCH_SIZE = 32

In [6]:
# preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [7]:
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [8]:
# load dataset
train_data = train_datagen.flow_from_directory(
    output_train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 2019 images belonging to 6 classes.


In [9]:
val_data = val_datagen.flow_from_directory(
    output_val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 508 images belonging to 6 classes.


## **3. Membangun Model CNN**
Model CNN dirancang dengan beberapa lapisan utama:
1. **Convolutional Layers**: Untuk mengekstraksi fitur dari gambar menggunakan filter yang bergerak di atas gambar.
2. **Pooling Layers**: Mengurangi dimensi data sambil mempertahankan informasi penting.
3. **Fully Connected Layers**: Lapisan akhir untuk menghasilkan prediksi berdasarkan fitur yang diekstraksi.

Arsitektur model disusun dengan filter bertahap yang meningkatkan kompleksitas, diikuti dengan dropout untuk mencegah overfitting.

---

In [10]:
# model CNN
model = Sequential([
    Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 6 kategori sampah
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## **4. Melatih Model**
Pada tahap ini:
- Dataset yang telah diproses digunakan untuk melatih model.
- Callback seperti **Early Stopping** digunakan untuk menghentikan pelatihan jika validasi loss tidak membaik dalam beberapa epoch, sehingga mencegah overfitting.
- Proses pelatihan dilakukan untuk beberapa epoch hingga model mencapai akurasi yang memadai.

---

In [11]:
# Callback untuk menghentikan pelatihan jika validasi tidak meningkat
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping]
)

C:\Users\musta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 26s 383ms/step - accuracy: 0.2398 - loss: 1.7062 - val_accuracy: 0.3819 - val_loss: 1.5152
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 142ms/step - accuracy: 0.3646 - loss: 1.5111 - val_accuracy: 0.4173 - val_loss: 1.4377
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 144ms/step - accuracy: 0.4067 - loss: 1.4073 - val_accuracy: 0.4016 - val_loss: 1.4138
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 144ms/step - accuracy: 0.4276 - loss: 1.3864 - val_accuracy: 0.3760 - val_loss: 1.4536
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 146ms/step - accuracy: 0.4130 - loss: 1.3882 - val_accuracy: 0.4272 - val_loss: 1.3326
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 144ms/step - accuracy: 0.4578 - loss: 1.3418 - val_accuracy: 0.4429 - val_loss: 1.3268
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 145ms/step - accuracy: 0.4787 - loss: 1.3241 - val_accuracy: 0.4449 - val_loss: 1.3291
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 146ms/step - accuracy: 0.4785 - loss: 1.2809 - val_accuracy: 0

## **5. Evaluasi Model**
Evaluasi dilakukan untuk mengukur performa model pada data validasi. Metode ini membantu memahami seberapa baik model dapat menggeneralisasi data yang tidak dilihat selama pelatihan. Metode evaluasi memberikan nilai:
- **Loss**: Tingkat kesalahan model.
- **Accuracy**: Persentase prediksi yang benar.

---

In [12]:
# Evaluasi model
loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.6124 - loss: 1.0211
Validation Loss: 0.9619932770729065
Validation Accuracy: 0.6318897604942322


# input

In [13]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

## **6. Prediksi Gambar**
Setelah model dilatih, model digunakan untuk memprediksi kategori dari gambar baru. Langkah-langkah prediksi meliputi:
1. **Preprocessing Gambar Baru**: Gambar diubah ke ukuran yang sesuai dan di-normalisasi.
2. **Prediksi**: Gambar yang telah diproses diberikan ke model untuk menghasilkan prediksi kategori.

Prediksi ini membantu dalam memahami bagaimana model mengklasifikasikan gambar di luar data pelatihan.

---

In [14]:
def preprocess_image(image_path, target_size):
    #gambar dari path
    img = load_img(image_path, target_size=target_size) 
    img_array = img_to_array(img) 
    img_array = img_array / 255.0  
    img_array = np.expand_dims(img_array, axis=0) 
    return img_array

In [15]:
image_path = 'kaleng.jpg'
processed_image = preprocess_image(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
predictions = model.predict(processed_image)
predicted_class = np.argmax(predictions, axis=1) 
class_labels = ['Cardboard', 'Glass', 'Metal', 'Paper', 'Plastic', 'Trash'] 
print(f"Prediksi: {class_labels[predicted_class[0]]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Prediksi: Metal
